In [34]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os

In [35]:
def load_data(x, y=None):
    data = []
    for file in x:
        data.append(pd.read_csv(file, index_col=False, 
                            usecols=['speed','flow_per_lane','before_on_ramp','after_on_ramp',
                                     'before_off_ramp','after_off_ramp']).to_numpy())
    if y is not None:
        labels = []
        for file in y:
            labels.append(pd.read_csv(file, index_col=False, header=None).to_numpy().transpose()[0])
        return np.array(data), np.array(labels)
    else:
        return np.array(data)

In [39]:
model = keras.models.load_model("C:/Aimsun Projects/Calibration Using Neural Networks/normalized_minmax_model.h5")
features_min = np.load("C:/Aimsun Projects/Calibration Using Neural Networks/normalized_minmax_model_min_x.npy")
features_max = np.load("C:/Aimsun Projects/Calibration Using Neural Networks/normalized_minmax_model_max_x.npy")
labels_min = np.load("C:/Aimsun Projects/Calibration Using Neural Networks/normalized_minmax_model_min_y.npy")
labels_max = np.load("C:/Aimsun Projects/Calibration Using Neural Networks/normalized_minmax_model_max_y.npy")

W0726 16:59:35.475966 27792 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.


In [40]:
model.compile(loss='mse', 
                          optimizer=tf.train.RMSPropOptimizer(0.001), 
                          metrics=['mae'])

In [6]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 972)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 128)               124544    
_________________________________________________________________
dense_21 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_22 (Dense)             (None, 9)                 1161      
Total params: 142,217
Trainable params: 142,217
Non-trainable params: 0
_________________________________________________________________


In [45]:
features, labels = load_data(["C:/Aimsun Projects/Calibration Using Neural Networks/dataset/x25.csv"],
                            ["C:/Aimsun Projects/Calibration Using Neural Networks/dataset/y25.csv"])

In [42]:
features.shape

(1, 162, 6)

In [46]:
labels[0]

array([  0.4,   0.6,   2.9,   4.5,  64. ,   8. , 170. , 370. , 370. ])

In [47]:
features_norm = features
features_norm[:,:,:2] = (features[:,:,:2] - features_min) / (features_max - features_min)
model.predict(features_norm) * (labels_max - labels_min) + labels_min

array([[ 3.27941716e-01, -2.57865667e-01,  3.16032660e+00,
         3.14722419e+00,  5.11154950e+01,  3.51987422e+01,
         1.75051796e+02,  5.51118040e+02,  5.36905885e+02]])